## Publish a Data Product
There are several steps to publish a data product to BDC:

1. Install SAP SDK
2. Create secret scope 
3. Create client
4. Create share
5. Add recipient to share
6. Create ORD object
7. Create CSN object
8. Publish data product


### 1. Install and Load Packages
To be able to share the enhanced data products back to SAP Business Data Cloud, we need the `SAP SDK`. 

In [0]:
%pip install sap-bdc-connect-sdk
%pip install --upgrade pydantic
%restart_python

In [0]:
pip show sap-bdc-connect-sdk

### 2. Create secret scope

This step has already been done by the administrator to facilitate the sharing process. However, if you need to do it on you own you can follow the steps described here to create a secret scope:

To create a secret scope you can either use the following URL https://<databricks-instance>#secrets/createScope. Replace <databricks-instance> with the workspace URL of your Databricks deployment.

Alternatively, you can run the following command in the terminal by clicking on the terminal icon on the lower right corner: `databricks secrets create-scope sap-bdc-connect-sdk`. 

The secret scope only has to be created once and can be made accessible to all workspace users by either toggling `manage principal` to `all workspace users` or via the terminal using the following command `databricks secrets put-acl sap-bdc-connect-sdk users READ`. To check whether the assignment worked, you can then use the command `databricks secrets list-acls sap-bdc-connect-sdk`.

A full explanation can be found here https://docs.databricks.com/aws/en/security/secrets/example-secret-workflow

### 3. Create a Client
The `DatabricksClient` receives dbutils as a parameter, which is a Databricks utility that can be used inside the Databricks notebooks.

The `BdcConnectClient` receives the DatabricksClient as a parameter to get information from the Databricks environment (e.g. secrets, api_token, workspace_url_base)

In [0]:
from bdc_connect_sdk.auth import BdcConnectClient
from bdc_connect_sdk.auth import DatabricksClient
from bdc_connect_sdk.utils import csn_generator
import json

databricks_client = DatabricksClient(dbutils)
bdc_connect_client = BdcConnectClient(databricks_client)

### 4. Create or Update Share
A share is a mechanism for distributing and accessing data across different systems. Creating or updating a share involves including specific attributes, such as @openResourceDiscoveryV1, in the request body, aligning with the Open Resource Discovery protocol. This procedure ensures that the share is properly structured and described according to specified standards, facilitating effective data sharing and management. This is done in step 5.

In the following code you'll need to set the parameters for the share:
- `<SHARE_NAME>`
- `<SHORT_DESCRIPTION>`
- `<LONG_DESCRIPTION>`
- `<DATA_PRODUCT_NAME>`

To be able to share a data product, the following requirements should be fulfilled:
- table has primary keys 
- DeletionVectors = disabled
- ChangeDataFeed = enabled 

We again set the location to the according catalog and schema.

In [0]:
%sql
-- CREATE CATALOG IF NOT EXISTS <CATALOG_NAME>;
SET CATALOG uc_delayed_payment;
-- CREATE SCHEMA IF NOT EXISTS <SCHEMA_NAME>;
USE SCHEMA grp1;

In [0]:
%sql
CREATE SHARE IF NOT EXISTS delayed_payment_grp1;
ALTER SHARE delayed_payment_grp1 ADD TABLE delay_prediction_dataset_shap WITH HISTORY;

### 5. Add Recipient to Share

To be able to share the data product to BDC we need to select `sap-business-data-cloud` as the recipient for the share we've just created. This can be either done by heading over to the **Unity Catalog** > **Delta Sharing** > **Shared by me** > *Select Share* > **Recipient** > **Add Recipients** > *Select sap-business-data-cloud*

Alternatively, the recipient can be added using the following query. Please fill in the `<SHARE_NAME>` and the `<RECIPIENT_NAME>`. 

In [0]:
%sql
GRANT SELECT ON SHARE delayed_payment_grp1 TO RECIPIENT `sap-business-data-cloud`;

### 6. Create ORD object 



In [0]:
with BdcConnectClient(DatabricksClient(dbutils)) as bdc_connect_client:
    share_name = "delayed_payment_grp1"

try:
    share_body = {
        "type": "REMOTE_SHARE",
        "provider": {
            "type": "FEDERATION",
            "name": "databricks"
        },
        "@openResourceDiscoveryV1": {
            "title": "Delayed Payment Prediction Data Product",
            "shortDescription": "Data asset for payment predictions",
            "description": "This data product contains the data used for the payment prediction model."
        }
    }

    share_request_body = json.dumps(share_body)

    catalog_response = bdc_connect_client.create_or_update_share(
        share_name,
        share_request_body
    )
except Exception as ex:
    print(f"Exception when creating or updating share(name={share_name}): {ex}\n")

### 7. Create CSN object

The CSN serves as a standardized format for configuring and describing shares within a network. To create or update the CSN for a share, it's advised to prepare the CSN content in a separate file and include this content in the request body. This approach ensures accuracy and compliance with the CSN interoperability specifications, facilitating consistent and effective share configuration across systems.

In the following code you'll need to set the parameter `<SHARE_NAME>` for the share.

In [0]:
with BdcConnectClient(DatabricksClient(dbutils)) as bdc_connect_client:
    share_name = "delayed_payment_grp1"

try:
    csn_schema = csn_generator.generate_csn_template(share_name)
    csn_schema_string = json.dumps(csn_schema)
    
    csn_response = bdc_connect_client.create_or_update_share_csn(
        share_name,
        csn_schema_string
    )
except Exception as ex:
    print(f"Exception when creating or updating CSN for share(name={share_name}): {ex}\n")

### 8. Publish a Data Product
A Data Product is an abstraction that represents a type of data or data set within a system, facilitating easier management and sharing across different platforms. It bundles resources or API endpoints to enable efficient data access and utilization by integrated systems. Publishing a Data Product allows these systems to access and consume the data, ensuring seamless communication and resource sharing.

In the following code you'll need to set the parameter `<SHARE_NAME>` for the share.

In [0]:
with BdcConnectClient(DatabricksClient(dbutils)) as bdc_connect_client:
    share_name = "delayed_payment_grp1"

try:
    publish_response = bdc_connect_client.publish_data_product(
        share_name,
    )
except Exception as ex:
    print(f"Exception when publishing data product for share(name={share_name}): {ex}\n")